In [1]:
import os
import time
import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use('dark_background')

import jax
import jax.numpy as jnp

jax.config.update("jax_enable_x64", True)


# Host CPUs

# os.environ['XLA_FLAGS'] = '--xla_force_host_platform_device_count=16'

# os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'false'
# os.environ['XLA_PYTHON_CLIENT_ALLOCATOR'] = 'platform'

# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = ", ".join(map(str, range(0, 1)))

jax.devices()

[StreamExecutorGpuDevice(id=0, process_index=0, slice_index=0),
 StreamExecutorGpuDevice(id=1, process_index=0, slice_index=0),
 StreamExecutorGpuDevice(id=2, process_index=0, slice_index=0),
 StreamExecutorGpuDevice(id=3, process_index=0, slice_index=0),
 StreamExecutorGpuDevice(id=4, process_index=0, slice_index=0),
 StreamExecutorGpuDevice(id=5, process_index=0, slice_index=0),
 StreamExecutorGpuDevice(id=6, process_index=0, slice_index=0),
 StreamExecutorGpuDevice(id=7, process_index=0, slice_index=0)]

In [234]:
import dimod

import networkx as nx

from tabu import TabuSampler

from neal.sampler import SimulatedAnnealingSampler

from dwave.samplers import SteepestDescentSolver
# from dwave.samplers import SimulatedAnnealingSampler


# solver = SteepestDescentSolver()
# solver = SimulatedAnnealingSampler()
solver = TabuSampler()

# x, y = dimod.Binaries(['x', 'y'])

# qubos = [x + y - 2.5 * x * y, 
         # x + y + 2.5 * x * y]

# graph = dimod.to_networkx_graph(qubos[1])

# nx.draw_networkx(graph)

In [235]:
# qubo_0 = {(0, 0): 1,
#           (0, 1): 2,
#           (1, 0): 2,
#           (1, 1): 1}

# qubo_1 = {(0, 0): 1,
#           (0, 1): -2,
#           (1, 0): -2,
#           (1, 1): 1}

qubo_0 = jnp.array([[1, 2],
                    [2, 1]])

qubo_1 = jnp.array([[1, -2],
                    [-2, 1]])

qubos = jnp.array([qubo_0, qubo_1])

sampleset = solver.sample_qubo(Q=qubo_0)

print(sampleset)

sampleset = solver.sample_qubo(Q=qubo_1)

print(sampleset)

   0  1 energy num_oc. num_re.
0  0  0    0.0       1       1
['BINARY', 1 rows, 1 samples, 2 variables]
   0  1 energy num_oc. num_re.
0  1  1   -2.0       1       2
['BINARY', 1 rows, 1 samples, 2 variables]


In [236]:
def custom_sample(qubo):
    
    sampleset = solver.sample_qubo(Q=qubo)
    
    sample = sampleset.first.sample
    
    # sample = qubo
    
    return sample

In [237]:
pmapped_sample = jax.pmap(
    custom_sample,
    # jitted_monte_carlo_mean,
    # static_broadcasted_argnums=(6, 7, 8),
    # in_axes=(None, None, None, None, None, 0, None, None, None, 0)
)

vmapped_sample = jax.vmap(
    custom_sample,
    # jitted_monte_carlo_mean,
    # static_broadcasted_argnums=(6, 7, 8),
    # in_axes=(None, None, None, None, None, 0, None, None, None, 0)
)

In [238]:
for qubo in qubos:
    
    sample = custom_sample(qubo)
    
    print(sample)

{0: 0, 1: 0}
{0: 1, 1: 1}


In [239]:
samples = vmapped_sample(qubos)

print(samples)

TracerArrayConversionError: The numpy.ndarray conversion method __array__() was called on the JAX Tracer object Traced<ShapedArray(int64[2,2])>with<BatchTrace(level=1/0)> with
  val = Array([[[ 1,  2],
        [ 2,  1]],

       [[ 1, -2],
        [-2,  1]]], dtype=int64)
  batch_dim = 0
This BatchTracer with object id 140146658698400 was created on line:
  /tmp/ipykernel_34880/1496653291.py:1 (<module>)
See https://jax.readthedocs.io/en/latest/errors.html#jax.errors.TracerArrayConversionError